# Multisensor fusion 
- Experiment： Maraging Steel 300
- Experiment number (single bead wall samples): 21-26
- Recorded data: position, veolocity, coaxial ccd images, acoustic data
- Defect generated: keyhole pores, cracks, defect-free

### Notebook 2: training with only vision dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

import os
import argparse
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torch.utils.data import Subset
import pandas as pd
import numpy as np

from models import *
from multimodaldataset import MultimodalDataset, LDEDAudioDataset, LDEDVisionDataset
from utils import progress_bar


Multimodal_dataset_PATH = os.path.join("C:\\Users\\Asus\\OneDrive_Chen1470\\OneDrive - Nanyang Technological University\\Dataset\\Multimodal_AM_monitoring\\LDED_Acoustic_Visual_Dataset")
CCD_Image_30Hz_path = os.path.join(Multimodal_dataset_PATH, 'Coaxial_CCD_images_30Hz')
Audio_segmented_30Hz_PATH = os.path.join(Multimodal_dataset_PATH, 'Audio_signal_all_30Hz')
Audio_raw_seg_PATH = os.path.join(Audio_segmented_30Hz_PATH, 'raw')
Audio_equalized_seg_PATH = os.path.join(Audio_segmented_30Hz_PATH, 'equalized')
Audio_bandpassed_seg_PATH = os.path.join(Audio_segmented_30Hz_PATH, 'bandpassed')
Audio_denoised_seg_PATH = os.path.join(Audio_segmented_30Hz_PATH, 'denoised')
AUDIO_DIR = Audio_denoised_seg_PATH
VISON_DIR = CCD_Image_30Hz_path

torch.manual_seed(0)

ANNOTATIONS_FILE = os.path.join(Multimodal_dataset_PATH, "vision_acoustic_label.csv")

classes = ('Defect-free', 'Cracks', 'Keyhole pores', 'Laser-off', 'Laser-start')
SAMPLE_RATE = 44100
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 15

### Use GPU if possible

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [3]:
annotations_df = pd.read_csv(ANNOTATIONS_FILE)
annotations_df.head()


sample_index    audio_file_name           image_file_name  class_ID  \
0             0  Sample22_0000.wav  sample22_frame000000.jpg         3   
1             1  Sample22_0001.wav  sample22_frame000001.jpg         3   
2             2  Sample22_0002.wav  sample22_frame000002.jpg         3   
3             3  Sample22_0003.wav  sample22_frame000003.jpg         3   
4             4  Sample22_0004.wav  sample22_frame000004.jpg         3   

  class_name  
0  Laser-off  
1  Laser-off  
2  Laser-off  
3  Laser-off  
4  Laser-off

In [4]:
# Get the labels and count the number of samples for each class
labels = annotations_df['class_ID'].values
label_counts = np.unique(labels, return_counts=True)[1]
print (label_counts)

[1889 2413  844  389   80]


In [5]:
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')

## Define training, testing (evaluation) function

In [18]:
def train_single_epoch(model, epoch, trainloader, loss_fn, optimizer, device):
    '''
    Function for the training single epoch of the training loop
    '''
    print('\nEpoch: %d' % epoch)
    model.train() # training mode
    running_loss = 0
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # for inputs, targets in trainloader:
        # print(type(targets))
        inputs, targets = inputs.to(device), targets.to(device)
        # calculate loss (forward pass)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        running_loss += loss.item() * inputs.size(0)
        # backpropagate error and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print(f"loss: {loss.item()}")
    epoch_loss = running_loss / len(trainloader.dataset)
    print("---------------------------")
    return model, optimizer, epoch_loss

In [7]:
def test_single_epoch(model, epoch, testloader, loss_fn, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            ## forward pass and calculate loss
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            # test_loss += loss.item()
            test_loss += loss.item()
            # _, predicted = outputs.max(1)
            # total += targets.size(0)* inputs.size(0)
            # correct += predicted.eq(targets).sum().item()

            # progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    # acc = 100.*correct/total
    # if acc > best_acc:
    #     print('Saving..')
    #     state = {
    #         'model': model.state_dict(),
    #         'acc': acc,
    #         'epoch': epoch,
    #     }
    #     if not os.path.isdir('checkpoint'):
    #         os.mkdir('checkpoint')
    #     torch.save(state, './checkpoint/ckpt.pth')
    #     best_acc = acc
    
    epoch_loss = test_loss / len(testloader.dataset)

In [8]:
def training_loop(model, loss_fn, optimizer, train_loader, valid_loader, epochs, scheduler, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train_single_epoch(model, epoch, train_loader, loss_fn, optimizer, device)
        train_losses.append(train_loss)

        # validation
        # with torch.no_grad():
        model, valid_loss = test_single_epoch(model, epoch, valid_loader, loss_fn, device)
        valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')
        
        scheduler.step()

    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

## Preparing Dataset and DataLoader

In [9]:
#------ transformation------
train_transforms=transforms.Compose([
    torchvision.transforms.Resize((32,32)), # original image size: (640,480)
    transforms.RandomHorizontalFlip(),  # data augmentation
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=90),
    transforms.ToTensor(),
    transforms.Normalize(mean=[136.20371045013258], std=[61.9731240029325]),
])

val_transforms=transforms.Compose([
    torchvision.transforms.Resize((32,32)), # original image size: (640,480)
    transforms.ToTensor(),
    transforms.Normalize(mean=[136.20371045013258], std=[61.9731240029325]),
])

In [10]:
#------ data loader------
# create the dataset for all samples
visiondataset = LDEDVisionDataset(ANNOTATIONS_FILE,
                                  VISON_DIR,
                                  train_transforms,
                                  device)
print ("length of the total dataset:" + str(len(visiondataset)))

length of the total dataset:163


### Dealing with Imbalanced dataset: stratified sampling

In [11]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# dealing with dataset imbalance: stratified sampling
train_idx, val_idx, test_idx = [], [], []
for class_idx, class_count in enumerate(label_counts):
    indices = np.where(labels == class_idx)[0]
    # Shuffle the indices for this class
    np.random.shuffle(indices)

    train_split = int(train_ratio * class_count)
    val_split  = int(val_ratio  * class_count) + train_split

    train_idx.extend(indices[:train_split])
    val_idx.extend(indices[train_split:val_split])
    test_idx.extend(indices[val_split:])

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)
test_sampler = SubsetRandomSampler(test_idx)

In [12]:
# Create the DataLoaders for each dataset
train_loader = DataLoader(LDEDVisionDataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(LDEDVisionDataset, batch_size=BATCH_SIZE, sampler=val_sampler)
test_loader = DataLoader(LDEDVisionDataset, batch_size=BATCH_SIZE, sampler=test_sampler)

### Dealing with Imbalanced dataset: stratified sampling (method 2)

In [13]:
# Split the data into train and validation datasets
# train_annotations, val_annotations = train_test_split(annotations_df, test_size=0.2)
# create the StratifiedShuffleSplit object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# split the indices of annotations into train and test sets
train_indices, test_indices = next(sss.split(annotations_df, labels))

# split the data into train and test sets
train_annotations = annotations_df.iloc[train_indices, :]
test_annotations = annotations_df.iloc[test_indices, :]

# Create the train and validation datasets
train_dataset = LDEDVisionDataset(train_annotations,
                                  image_path = VISON_DIR,
                                  image_transformation=train_transforms,
                                  device=device)

val_dataset = LDEDVisionDataset(test_annotations,
                                image_path=VISON_DIR,
                                image_transformation=val_transforms,
                                device=device)

# # Create train and val dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("length of the train dataset:" +  str(len(train_dataset)))
print("length of the val dataset:" +  str(len(val_dataset)))

length of the train dataset:4492
length of the val dataset:1123


## Training the model

In [14]:
resume = False
start_epoch = 0
best_acc = 0

In [19]:
# -----Model---------------
print('==> Building model..')
net = VGG('VGG19')
# net = LeNet() 
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
# net = SimpleDLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

loss_fn = nn.CrossEntropyLoss()
# Note: weight_decay in pytorch is L2 regularization
# optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE,
#                     momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# for epoch in range(start_epoch, start_epoch+EPOCHS):
#     train_single_epoch(net, epoch, train_dataloader, loss_fn, optimizer, device)
#     test_single_epoch(net, epoch, val_dataloader, loss_fn, device)
#     scheduler.step()

model, optimizer, _ = training_loop(net, loss_fn, optimizer, train_dataloader, val_dataloader, EPOCHS, scheduler, device)

==> Building model..

Epoch: 0
 [================================================================>]  Step: 442ms | Tot: 21s305ms | Loss: 4.893 | Acc: 35.018% (1573/449 141/141 
loss: 4.567245960235596
---------------------------


TypeError: cannot unpack non-iterable NoneType object